## Section 1.1

Problem 1 Code

In [ ]:
import numpy as np
from cil.framework import  AcquisitionGeometry, ImageData, AcquisitionData, ImageGeometry
from cil.plugins.astra.operators import ProjectionOperator
from cil.utilities.display import show2D

N = 128
angles = np.linspace(0,180, int(np.pi*N/2)+1)[:-1]
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(N, pixel_size=1.0)
ig = ag.get_ImageGeometry()
A = ProjectionOperator(ig, ag, device='gpu')
X = ig.allocate()
X.array[30,30] = 1

S = A.direct(X)
show2D(S)



Problem 2 Code

In [ ]:
from cil.plugins import TomoPhantom as cilTomoPhantom
Ns = [101, 201, 501, 1001]
Ps = []
for n in Ns:
    ig = ImageGeometry(voxel_num_x=n, 
                    voxel_num_y=n, 
                    voxel_size_x=1, 
                    voxel_size_y=1)
    Ps.append(cilTomoPhantom.get_ImageData(num_model=1, geometry=ig))
show2D(Ps)

Problem 3 Code

In [ ]:
angles = np.linspace(0,179,180)
n = 513

ig = ImageGeometry(voxel_num_x=n, 
                    voxel_num_y=n, 
                    voxel_size_x=1, 
                    voxel_size_y=1)

P = cilTomoPhantom.get_ImageData(num_model=1, geometry=ig)
show2D(P)

angles = np.linspace(0,179,180)
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(n, pixel_size=1.0)
A = ProjectionOperator(ig, ag, device='gpu')

S = A.direct(P)
show2D(S)


In [ ]:
from cil.utilities.display import show1D
angle_ids = np.arange(0,179,30)

S_slices = []
for angle_id in angle_ids:
    S_slices.append(S.get_slice(angle = angle_id))
show1D(S_slices)


In [ ]:
angles = np.linspace(0,179,1800)
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(n, pixel_size=1.0)

A = ProjectionOperator(ig, ag, device='gpu')
S = A.direct(P)
show2D(S)

angle_ids = np.arange(0,1799,300)
S_slices = []
for angle_id in angle_ids:
    S_slices.append(S.get_slice(angle = angle_id))
show1D(S_slices)

Problem 4 Code

In [ ]:
angles = np.arange(360)
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(n, pixel_size=1.0)

A = ProjectionOperator(ig, ag, device='gpu')
S = A.direct(P)

show2D(S)
print(S.shape)

S0 = S.array[:180,:]
S1 = np.flip(S.array[180:,:],1)
show2D([S0, S1, S0 - S1])

Problem 5

In [ ]:
N = 513
grid_mks = np.arange(N) - (N-1)/2
X, Y = np.meshgrid(grid_mks, grid_mks)
P = ((X**2 + Y**2)**(1/2) < (N-1)/3).astype(np.float32)
P = ImageData(P, geometry=ig)
angles = np.linspace(0,179,180)
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(n, pixel_size=1.0)
A = ProjectionOperator(ig, ag, device='gpu')
S = A.direct(P)
show2D([P,S])

Problem 6 Code

In [ ]:
N = 513
grid_mks = np.arange(N) - (N-1)/2
P = ((4*X**2 + Y**2)**(1/2) < (N-1)/3).astype(np.float32)
P = ImageData(P, geometry=ig)
S = A.direct(P)
show2D([P,S])

## Section 1.2

In [ ]:
N = 513
P1 = ((4*X**2 + Y**2)**(1/2) < (N-1)/3).astype(np.float32)
P1 = ImageData(P1, geometry=ig)
P2 = ((X**2 + 4*Y**2)**(1/2) < (N-1)/3).astype(np.float32)
P2 = ImageData(P2, geometry=ig)

a1 = 2.0
a2  = -3.0

P3 = P1.copy()
P3.array = a1*P1.array + a2*P2.array

show2D([P1, P2, P3])


S1 = A.direct(P1)
S2 = A.direct(P2)
S3 = A.direct(P3)
S4 = S3.copy()
S4.array = a1*S1.array + a2*S2.array
S_err = S3.copy()
S_err.array -= S4.array

show2D([S1, S2, S3, S4, S_err])

## Section 1.3

Problem 1

In [ ]:
N = 251
X = cilTomoPhantom.get_ImageData(num_model=1, geometry=ig)
show2D(X)

Problem 2

In [ ]:
angles = np.arange(360)/2
ag = AcquisitionGeometry.create_Parallel2D()\
                                .set_angles(angles)\
                                .set_panel(n, pixel_size=1.0)
A = ProjectionOperator(ig, ag, device='gpu')
Y = A.direct(X)
Y.array /= N
show2D(Y)

Problem 3

In [ ]:
I0 = 10**4
I = Y.copy()
I.array = I0*np.exp(-Y.array)

Y_ref = Y.copy()
Y_ref.array *= 0

I_ref = Y.copy()
I_ref.array = I0*np.exp(-Y_ref.array)
show2D([I, I_ref])

I_slice = I.get_slice(angle = 0)
I_ref_slice = I_ref.get_slice(angle = 0)

show1D([I_slice, I_ref_slice])

Problem 4

In [ ]:
I_noisy = np.random.poisson(I.array)
I_ref_noisy = np.random.poisson(I_ref.array)
show2D([I_noisy, I_ref_noisy])

show1D([I_noisy[0,:],I_ref_noisy[0,:]])

Problem 5

In [ ]:
Y_noisy = -np.log(I_noisy/I0)
show2D([Y, Y_noisy])

Problem 6

Repating for $I_0 = 10^6$

In [ ]:
I0 = 10**6
I = Y.copy()
I.array = I0*np.exp(-Y.array)

Y_ref = Y.copy()
Y_ref.array *= 0

I_ref = Y.copy()
I_ref.array = I0*np.exp(-Y_ref.array)
show2D([I, I_ref])

I_slice = I.get_slice(angle = 0)
I_ref_slice = I_ref.get_slice(angle = 0)

show1D([I_slice, I_ref_slice])
I_noisy = np.random.poisson(I.array)
I_ref_noisy = np.random.poisson(I_ref.array)
show2D([I_noisy, I_ref_noisy])

show1D([I_noisy[0,:],I_ref_noisy[0,:]])

Y_noisy = -np.log(I_noisy/I0)
show2D([Y, Y_noisy])

Repeating for $I_0 = 10^2$

In [ ]:
I0 = 10**2
I = Y.copy()
I.array = I0*np.exp(-Y.array)

Y_ref = Y.copy()
Y_ref.array *= 0

I_ref = Y.copy()
I_ref.array = I0*np.exp(-Y_ref.array)
show2D([I, I_ref])

I_slice = I.get_slice(angle = 0)
I_ref_slice = I_ref.get_slice(angle = 0)

show1D([I_slice, I_ref_slice])
I_noisy = np.random.poisson(I.array)
I_ref_noisy = np.random.poisson(I_ref.array)
show2D([I_noisy, I_ref_noisy])

show1D([I_noisy[0,:],I_ref_noisy[0,:]])

Y_noisy = -np.log(I_noisy/I0)
show2D([Y, Y_noisy])